In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk

# load the dataset
dataset = pd.read_csv('./data/train.csv')
dataset.head()

start = 15121
end = 18002

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    lem = WordNetLemmatizer()
    res = [lem.lemmatize(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, dataset.shape[0]):
    res.append(get_category(dataset['Page content'][i]))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
cats = res

In [16]:
from pytrends.request import TrendReq
from datetime import *
import time
import requests
import random
from termcolor import colored

In [5]:
def get_proxy_list():
    proxies = requests.get("https://proxy.rudnkh.me/txt")
    # print(res.text)
    proxy_list = ["http://"+proxy for proxy in proxies.text.split("\n")]
    proxy_list.remove("http://")
    random.shuffle(proxy_list)
    #print("Candidate: " + str(proxy_list))
    return proxy_list

# Test
# print(get_proxy_list())

In [21]:
def get_trend(word: str, t_start: str, t_end: str) -> list:
    search = word +" "+ t_start +" "+ t_end
    print(colored(search, "yellow"))
    g_trends = TrendReq(tz=360, timeout=(10,25), proxies=get_proxy_list(), retries=4, backoff_factor=0.1)
#     g_trends = TrendReq(hl='en-US', tz=360)
    g_trends.build_payload([word], timeframe = t_start+' '+t_end, geo='', gprop='')
    trend = g_trends.interest_over_time()
    if trend.size > 0:
        trend.columns = ['0', '1']
        #trend = trend.columns.get_loc()
        # trend.head()
        return trend['0'].values
    else: 
        print(word+" request fail.")
        return []

# test
# print(get_trend('NFV', '2013-06-20', '2013-07-20'))
# print(get_trend('NFV', '2013-06-20', '2013-07-20'))
# print(np.mean(get_trend('NFV', '2013-06-20', '2013-07-20'), axis=0))
# print(np.std(get_trend('NFV', '2013-06-20', '2013-07-20'), axis=0, ddof=1))
# print(np.std([1, 2, 3], ddof=1))

In [7]:
def get_date(text: str) -> datetime:
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    date = date.split()
    date = str(date[0] + ' ' + date[1])

    d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return d

In [24]:
from termcolor import colored

start = 18002
end = 18005
def trend_num(word: str, pub_time: datetime):
    #print(pub_time)
    y_start = str(pub_time+ timedelta(days=-15)).split(' ')[0]
    y_end = str(pub_time + timedelta(days=14)).split(' ')[0]
    for i in range(0, 20):
        try:
            y_trend = get_trend(word, y_start, y_end)
            break
        except Exception as e:
            print(e)
            print("Retry step: " + str(i))
    
    #print(y_trend)
    if len(y_trend) == 30:
        return y_trend
    else:
        return [0 for i in range(30)]
    
def trend_rate(word: str, pub_time: datetime):
    trend = trend_num(word, pub_time)
    now = np.sum(trend[15:])
    #time.sleep(1)
    past = np.sum(trend[:15])
    #time.sleep(1)
    if past > 0:
        return now/past
    else:
        return 2
    
def export(check_point):
    g_trend = pd.DataFrame(res)
    g_trend.columns = ["trend_rate"]
    ID = [i for i in range(start, check_point)]
    g_trend['Id'] = ID
    g_trend = g_trend.set_index('Id')
    g_trend.to_csv("trend_%s_%s.csv" % (str(start), str(check_point)))
    
res = []
for i in range(start, end):
    try:
        date = get_date(dataset["Page content"][i])
        tmp = [0]
        for cat in cats[i]:
            print(colored("Search: " + cat, "red"))
            tmp.append(trend_rate(cat, date))
        res.append(max(tmp))
        print(res)
        print(colored(str(i)+" done", "green"))
        if i%10 == 0:
            export(i+1)
            #time.sleep(30)
        if i == end-1:
            export(end)
    except Exception as e:
        print(e)
        export(i-1)
        print("Break at: " + str(i))
        n = 0
        print(np.mean(res))
        for i in range(start, start+len(res)):
            if (res[i] > float(np.mean(res))) and (1 == dataset["Popularity"][i]):
                n = n+1
            elif (res[i] < float(np.mean(res))) and (-1 == dataset["Popularity"][i]):
                n = n+1
            print(str(i) + ": \t" + str(dataset["Popularity"][i]) +"\t"+str(res[i]))
        print("accu: " + str(n/len(res)))

Search: tornado alley
tornado alley 2014-06-12 2014-07-11
Search: tornado
tornado 2014-06-12 2014-07-11
Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))
Retry step: 0
tornado 2014-06-12 2014-07-11
Search: tornado outbreak
tornado outbreak 2014-06-12 2014-07-11
Proxy error. Changing IP
Proxy error. Changing IP
Search: severe weather
severe weather 2014-06-12 2014-07-11
Search: climate
climate 2014-06-12 2014-07-11
The request failed: Google returned a response with code 429.
Retry step: 0
climate 2014-06-12 2014-07-11
Proxy error. Changing IP
Proxy error. Changing IP
Search: tornado wall
tornado wall 2014-06-12 2014-07-11
Search: u.s.
u.s. 2014-06-12 2014-07-11
Proxy error. Changing IP
Proxy error. Changing IP
Search: world
world 2014-06-12 2014-07-11
[0.9663526244952894]
1800

In [27]:
#res[1] = 2 0 o
# [1.3623188405797102]
# 18001 done
n = 0
print(np.mean(res))
for i in range(start, start+len(res)):
    if (res[i-18002] > float(np.mean(res))) and (1 == dataset["Popularity"][i]):
        n = n+1
    elif (res[i-18002] < float(np.mean(res))) and (-1 == dataset["Popularity"][i]):
        n = n+1
    print(str(i) + ": \t" + str(dataset["Popularity"][i]) +"\t"+str(res[i-18002]))
print("accu: " + str(n/len(res)))

1.6326325697256354
18002: 	1	0.9663526244952894
18003: 	-1	1.072570725707257
18004: 	-1	2.858974358974359
accu: 0.3333333333333333


In [ ]:
# def export():
#     g_trend = pd.DataFrame(res)
#     g_trend.columns = ["trend_rate"]
#     ID = [i for i in range(start, end)]
#     g_trend['Id'] = ID
#     g_trend = g_trend.set_index('Id')
#     g_trend.to_csv("trend.csv")

In [ ]:
"trend_%s_%s.csv" % (str(start), str(55))